In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import mean_squared_error as mse
from sklearn.svm import SVR
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error as mae

C:\Users\AURIMASSilva\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def impute_0(x):
    if pd.isnull(x):
        return 0.00
    else:
        return x

In [3]:
df = pd.read_csv('ready_returns.csv', header =None) # read BTC return data
df.columns = ['marge', 'returns'] # make columns nicer
df2 = pd.read_csv('sentiment_ready.csv') # read BTC sentiment data
df2.columns  = ['index', 'marge', 'sentiment'] # make columns nicer
df2 = df2.iloc[1:,:] # remove 2013-12-12 entry for sentiment
df3 = pd.merge(df,df2, how='outer', on='marge') # merge sentiment and btc return data
assert len(df) == len(df3)   # check if all dates are on
df3 = df3[['marge', 'returns', 'sentiment']] # leave relevant oclumns
df3.columns = ['date', 'returns', 'sentiment'] # change columns
df3['sentiment'] = df3['sentiment'].apply(lambda x : impute_0(x)) # impute missing values with 0

In [5]:
# additional btc price data
df_price = pd.read_csv('lol.csv')
df_price = df_price[1:]
df3['price'] = df_price.values

In [6]:
sentiment = df3['sentiment'].values
returns = df3['returns'].values
assert len(sentiment) == len(returns)

In [7]:
returns

array([ 2.30633546,  4.01737101,  2.03516993, ..., -8.07753641,
       -0.6181936 , -2.57139795])

In [8]:
sentiment

array([0.37426328, 0.25697687, 0.2750259 , ..., 0.41973929, 0.34067555,
       0.29402947])

In [9]:


data_gen = TimeseriesGenerator(np.append(sentiment,0.00), np.append(sentiment,0.00),
                                       length= 10, sampling_rate=1,
                                       batch_size=1)

# get features and targets from generated batches
features_sentiment = []
targets_sentiment = []
i = 0
while True:
    try:
        features_sentiment.append(list(data_gen[i][0][0]))
        targets_sentiment.append(data_gen[i][1][0])
        i += 1
    except:
        break
data_gen = TimeseriesGenerator(np.append(returns,0.00), np.append(returns,0.00),
                                       length= 10, sampling_rate=1,
                                       batch_size=1)

# get features and targets from generated batches
features_returns = []
targets_returns = []
i = 0
while True:
    try:
        features_returns.append(list(data_gen[i][0][0]))
        targets_returns.append(data_gen[i][1][0])
        i += 1
    except:
        break
        
assert len(features_sentiment) == len(features_returns)
assert len(targets_sentiment) == len(targets_returns)

In [13]:
features_sentiment[1]

[0.2569768707482993,
 0.27502589928057536,
 0.3302259374999999,
 0.25859109311740885,
 0.28534702258726896,
 0.2777257683215129,
 0.1969468671679196,
 0.21032805611222435,
 0.2229191596638656,
 0.2801149717514127]

In [14]:
targets_sentiment[0]

0.2801149717514127

In [15]:
features_returns[1]

[4.0173710137412515,
 2.0351699298346237,
 4.9821672855188535,
 8.611883515026001,
 2.1166968388803777,
 -15.870301796934822,
 5.077306733167085,
 0.4912663755458499,
 2.5529603476371543,
 5.235584008843039]

In [17]:
targets_returns[0]

5.235584008843039